In [72]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split

In [75]:
data_dir = '10flowers'
image_size = (128, 128) # Images will be resized to 128 by 128
batch_size = 40
num_classes = 10

In [76]:
### DATA LOADING AND PREPROCESSING
# Load the data and labels
data = []
labels = []
class_names = sorted(os.listdir(os.path.join(data_dir))) # Read the class names
# print("Class name: ", class_names)

for class_name in class_names:
    class_dir = os.path.join(data_dir, class_name) # Path to directory
    for img_file in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_file) # Path to the image
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=image_size) # Load and resize the image
        img = tf.keras.preprocessing.image.img_to_array(img) # Convert the image to an array
        
        # Append the image and the class label to the respective lists
        data.append(img)
        labels.append(class_name)

        
data = np.array(data)
# print(len(data))

labels = np.array(labels)
# print("Labels: ", labels)

In [77]:
### DATA CONVERSION AND SPLITTING
### Convert class names to integer labels
# Create a mapping from class name to integer labels
label_to_index = {label: i for i, label in enumerate(class_names)}
print("label_to_index: ", label_to_index)

# Converts class names to integer labels
labels = np.array([label_to_index[label] for label in labels])
print("labels: ", labels)
print("labels length: ", len(labels))

# Split data into training, validation, and testing sets
# 70% will be used for training
x_train, x_temp, y_train, y_temp = train_test_split(data, labels, test_size=0.3, random_state=42)
print("x_train size: ", len(x_train))
print("x_temp size: ", len(x_temp))


# SPlit the data from above validations and testing (15% each now)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)
print("x_val size: ", len(x_val))
print("y_val size: ", len(y_val))


label_to_index:  {'Bluebell': 0, 'Crocus': 1, 'Daffodil': 2, 'Daisy': 3, 'Fritillary': 4, 'Iris': 5, 'Lily Valley': 6, 'Snowdrop': 7, 'Sunflower': 8, 'Tulip': 9}
labels:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5
 5 5

In [78]:
# Convert integer labels to one-hot encoded labels
# To respresent class labels as categorical vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

print("y_train: ", y_train)

y_train:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [79]:
### DATA NORMALIZATION
# The RGB channel values are in the [0, 255] range.
# This is not ideal for a neural network
# Normalize the pixel values to [0, 1]
x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test / 255.0

In [80]:
# Build the model with regularization
inputs = Input(shape=(image_size[0], image_size[1], 3))
x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d_9 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 30, 30, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 28, 28, 128)       7385

In [81]:
# Compile the model using Adam optimizer
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [82]:
# Train the model
epochs = 20
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          batch_size=batch_size,
          epochs=epochs)w

Epoch 1/20
14/14 [==============================] - 10s 587ms/step - loss: 2.2473 - accuracy: 0.1821 - val_loss: 1.8626 - val_accuracy: 0.3000
Epoch 2/20
14/14 [==============================] - 8s 558ms/step - loss: 1.6190 - accuracy: 0.4089 - val_loss: 1.6954 - val_accuracy: 0.4000
Epoch 3/20
14/14 [==============================] - 9s 632ms/step - loss: 1.2185 - accuracy: 0.5589 - val_loss: 1.3397 - val_accuracy: 0.5417
Epoch 4/20
14/14 [==============================] - 9s 641ms/step - loss: 0.8935 - accuracy: 0.6839 - val_loss: 1.2675 - val_accuracy: 0.6083
Epoch 5/20
14/14 [==============================] - 8s 555ms/step - loss: 0.6146 - accuracy: 0.7911 - val_loss: 1.0160 - val_accuracy: 0.7083
Epoch 6/20
14/14 [==============================] - 8s 541ms/step - loss: 0.5900 - accuracy: 0.8107 - val_loss: 1.2522 - val_accuracy: 0.6083
Epoch 7/20
14/14 [==============================] - 7s 535ms/step - loss: 0.3320 - accuracy: 0.8929 - val_loss: 1.0462 - val_accuracy: 0.6833
Epoch

In [83]:
# Save the model
from tensorflow.keras.models import load_model
model.save(os.path.join('models','TenClasses1.keras'))